In [1]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import pandas as pd
import random
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg


In [3]:
# Load CSV file from uploaded location
df = pd.read_csv(r"C:\Users\sdshe\Downloads\music_recommender-master\music_recommender-master\content based recommedation system\songdata.csv").sample(n=3000).drop(columns="link").reset_index(drop=True)
df['text'] = df['text'].str.replace(r'\n', '', regex=True)
df.head()


,artist,song,text
0,Red Hot Chili Peppers,Porcelain,Porcelain Are you wasting away in your skin ...
1,Morrissey,The Boy Racer,He's just too good-natured and He's got too m...
2,Slayer,Gemini,Endure the pain You know my name I am your s...
3,King Diamond,Follow The Wolf,"The morning slowly came, my life about to end ..."
4,Ozzy Osbourne,Thank God For The Bomb,Like moths to a flame Is man never gonna chan...


In [5]:
vectorizer = TfidfVectorizer(stop_words='english')
matrix = vectorizer.fit_transform(df['text'])
cos_sim = cosine_similarity(matrix)


In [7]:
recs = {}
for i in range(len(df)):
    indices = cos_sim[i].argsort()[:-51:-1]
    recs[df['song'][i]] = [
        (cos_sim[i][j], df['song'][j], df['artist'][j], df['text'][j][:160] + "...")
        for j in indices if i != j
    ]


In [9]:
THEMES = {
    "light": {
        "bg": "#fefefe", "fg": "#222", "text_bg": "#ffffff", "text_fg": "#111",
        "quote": "#666", "accent": "#4fc3f7", "hover": "#81d4fa"
    },
    "dark": {
        "bg": "#1e1e1e", "fg": "#fafafa", "text_bg": "#121212", "text_fg": "#c8e6c9",
        "quote": "#ffa726", "accent": "#7986cb", "hover": "#aab6fe"
    }
}


In [11]:
class MusicRecommender:
    def __init__(self, matrix):
        self.matrix = matrix

    def recommend(self, song, n=5):
        return self.matrix.get(song, [])[:n]


In [13]:
class MusicApp:
    def __init__(self, root, recommender, songs):
        self.root = root
        self.recommender = recommender
        self.songs = sorted(songs)
        self.results = []
        self.theme = "light"

        self.root.title("🎵 LyricWave – Pie Chart Edition")
        self.root.geometry("1000x700")
        self.root.resizable(False, False)

        self.build_ui()
        self.apply_theme()

    def make_button(self, parent, text, command):
        btn = tk.Label(parent, text=text, font=("Segoe UI", 10, "bold"), width=16,
                       relief="ridge", padx=5, pady=5, cursor="hand2", bd=1)
        btn.bind("<Button-1>", lambda e: command())
        btn.bind("<Enter>", lambda e: btn.config(bg=self.hover_color, fg=self.fg_color))
        btn.bind("<Leave>", lambda e: btn.config(bg=self.accent_color, fg=self.fg_color))
        return btn

    def build_ui(self):
        self.header = tk.Canvas(self.root, height=60, highlightthickness=0)
        self.header_text = self.header.create_text(
            500, 30, text="🎶 LyricWave: Discover with Visual Insight",
            font=("Segoe UI", 20, "bold"))
        self.header.pack(fill=tk.X)

        self.song_var = tk.StringVar()
        self.combo = ttk.Combobox(self.root, textvariable=self.song_var, values=self.songs, width=92)
        self.combo.pack(pady=12)

        btn_frame = tk.Frame(self.root)
        btn_frame.pack(pady=5)

        self.rec_btn = self.make_button(btn_frame, "✨ Recommend", self.recommend)
        self.exp_btn = self.make_button(btn_frame, "💾 Export", self.export)
        self.res_btn = self.make_button(btn_frame, "🔄 Reset", self.reset)
        self.theme_btn = self.make_button(btn_frame, "🌓 Toggle Theme", self.toggle_theme)

        for btn in [self.rec_btn, self.exp_btn, self.res_btn, self.theme_btn]:
            btn.pack(side=tk.LEFT, padx=8, pady=5)

        self.output = tk.Text(self.root, height=16, width=114, font=("Consolas", 10), wrap=tk.WORD)
        self.output.pack(padx=20, pady=10)

        self.chart_frame = tk.Frame(self.root)
        self.chart_frame.pack(pady=5)

        self.quote = tk.Label(self.root, font=("Segoe UI", 10, "italic"))
        self.quote.pack()

    def apply_theme(self):
        t = THEMES[self.theme]
        self.accent_color = t["accent"]
        self.hover_color = t["hover"]
        self.fg_color = t["fg"]

        self.root.config(bg=t["bg"])
        self.header.config(bg=t["accent"])
        self.header.itemconfig(self.header_text, fill=self.fg_color)
        self.output.config(bg=t["text_bg"], fg=t["text_fg"], insertbackground=t["fg"])
        self.quote.config(bg=t["bg"], fg=t["quote"])
        self.combo.configure(background="white" if self.theme == "light" else "#333")

        for btn in [self.rec_btn, self.exp_btn, self.res_btn, self.theme_btn]:
            btn.config(bg=self.accent_color, fg=self.fg_color)

    def toggle_theme(self):
        self.theme = "dark" if self.theme == "light" else "light"
        self.apply_theme()
        self.quote.config(text=random.choice([
            "🌒 Dark or light—your vibe, your choice.",
            "🎨 Theme switched. Let’s spin new sounds.",
            "🌗 Bright minds shine in every palette."
        ]))

    def recommend(self):
        self.output.delete("1.0", tk.END)
        for w in self.chart_frame.winfo_children():
            w.destroy()

        song = self.song_var.get().strip()
        if song not in self.songs:
            messagebox.showwarning("Not Found", "Please select a valid song.")
            return

        now = datetime.now().strftime("%Y-%m-%d %H:%M")
        self.results = self.recommender.recommend(song)

        self.output.insert(tk.END, f"\n🎼 Top Matches for: “{song}”\n📅 {now}\n\n")
        for idx, (score, title, artist, snippet) in enumerate(self.results, 1):
            emoji = "🔥" if score > 0.5 else "🎵"
            self.output.insert(tk.END,
                f"{emoji} {idx}. {title} by {artist}\n"
                f"    ➤ Score: {score:.3f}\n"
                f"    ➤ Preview: “{snippet}”\n\n"
            )

        self.quote.config(text=random.choice([
            "🎧 'This chart? A new way to feel sound.'",
            "📊 'Lyrics, visualized. Music, quantified.'",
            "🎙️ 'Insight turns tunes into revelations.'"
        ]))

        self.show_pie_chart(song)

    def show_pie_chart(self, song):
        if not self.results: return
        labels = [f"{title}" for _, title, _, _ in self.results]
        sizes = [score for score, _, _, _ in self.results]
        fig, ax = plt.subplots(figsize=(5.2, 4.5), dpi=100)
        ax.pie(sizes, labels=labels, startangle=140, autopct='%1.1f%%', colors=plt.cm.Set3.colors)
        ax.set_title(f"Similarity Distribution for “{song}”", fontsize=10)
        fig.tight_layout()

        canvas = FigureCanvasTkAgg(fig, master=self.chart_frame)
        canvas.draw()
        canvas.get_tk_widget().pack()

    def export(self):
        if not self.results:
            messagebox.showinfo("Nothing Yet", "Please run a recommendation first.")
            return
        path = filedialog.asksaveasfilename(defaultextension=".txt",
                    filetypes=[("Text Files", "*.txt")])
        if path:
            with open(path, "w", encoding="utf-8") as f:
                f.write(f"🎧 Playlist for '{self.song_var.get()}'\n\n")
                for idx, (score, title, artist, snippet) in enumerate(self.results, 1):
                    f.write(f"{idx}. {title} by {artist}\n")
                    f.write(f"   Score: {score:.3f}\n")
                    f.write(f"   Sample: {snippet}\n\n")
            messagebox.showinfo("Exported", "Playlist saved successfully!")

    def reset(self):
        self.song_var.set("")
        self.output.delete("1.0", tk.END)
        for widget in self.chart_frame.winfo_children():
            widget.destroy()
        self.quote.config(text="")
        self.results = []


In [15]:
root = tk.Tk()
recommender = MusicRecommender(recs)
app = MusicApp(root, recommender, df['song'].unique())
root.mainloop()
